In [1]:
import subprocess
import os
import time
import numpy as np
import pandas as pd
import datetime
import spotpy
import xarray as xr
import hydroeval as he
#from configparser import ConfigParser

In [ ]:
class cosipy_model(object):

    def __init__(self): #self,startTime,endTime
        #self.startTime= startTime
        #self.endTime = endTime
        
        return    
    
    def get_obs(self):
        
        #function to translate from cosipy format to spotpy format
        def eval_to_cosipy(stakes_path):

            #if dtype == 'obs':
            stakes_data = pd.read_csv(stakes_path,index_col='TIMESTAMP')
            #elif dtype == 'sim':
            #    stakes_data = pd.read_csv(stakes_path,index_col='Unnamed: 0')

            s_names = []
            all_stakes = []
            for i in range(stakes_data.shape[0]):
                for j in range(stakes_data.shape[1]):
                    stake_name = str(stakes_data.columns[j])+'_'+str(i)
                    s_names.append(stake_name)
                    all_stakes.append(stakes_data.iloc[i,j])
                    #print(stake_name)

            data = pd.DataFrame(all_stakes,s_names)

            return data
        
        obs_path = '/home/phi/Documents/Mocho-COSIPY/input/stakes/stakes_data_2022-2023.csv' #TOKEN
        obs = eval_to_cosipy(obs_path)
        obs = obs.iloc[:,0].values
        
        self.observations = obs

        return 

    def run_cosipy(self,albedo_fresh_snow=None,albedo_firn=None,albedo_ice=None,
                   albedo_mod_snow_aging=None,albedo_mod_snow_depth=None,
                   roughness_fresh_snow=None,roughness_ice=None,
                   roughness_firn=None):
        
        
        #function to translate from cosipy format to spotpy format
        def eval_to_cosipy(stakes_path):

            #if dtype == 'obs':
            stakes_data = pd.read_csv(stakes_path,index_col='TIMESTAMP')
            #elif dtype == 'sim':
            #    stakes_data = pd.read_csv(stakes_path,index_col='Unnamed: 0')

            s_names = []
            all_stakes = []
            for i in range(stakes_data.shape[0]):
                for j in range(stakes_data.shape[1]):
                    stake_name = str(stakes_data.columns[j])+'_'+str(i)
                    s_names.append(stake_name)
                    all_stakes.append(stakes_data.iloc[i,j])
                    #print(stake_name)

            data = pd.DataFrame(all_stakes,s_names)

            return data
        

        def change_param_values(constant_path,var_names,values):

            os.chdir('/home/phi/Documents/COSIPY/cosipy-master/')

            # read constant.py lines
            with open(constant_path, 'r') as f:
                lines = f.readlines()

            # Modify the var lines
            for v in range(len(var_names)):
                for i, line in enumerate(lines):

                    if line.strip().startswith(var_names[v]):
                        lines[i] = f"{var_names[v]} = {repr(values[v])}\n"
                        break
                else:
                    # Print error
                    print(f"The var '{var_names[v]}' doesn't exist")
                    return

            # write the edited value
            with open(constant_path, 'w') as f:
                f.writelines(lines)

        #Read constant.py file (document with parameters to optimize)
        constantpy_path = '/home/phi/Documents/COSIPY/cosipy-master/constants.py'
        
        params_names = ['albedo_fresh_snow','albedo_firn','albedo_ice','albedo_mod_snow_aging','albedo_mod_snow_depth',
                        'roughness_fresh_snow','roughness_ice','roughness_firn']
        params_values = [albedo_fresh_snow,albedo_firn,albedo_ice,albedo_mod_snow_aging,albedo_mod_snow_depth,
                         roughness_fresh_snow,roughness_ice,roughness_firn] #vector with param values        
        
        #Change the param values in the config file THE MAGIC!!!!
        change_param_values(constantpy_path,params_names,params_values)
        
        #run sphy
        #os.chdir('/home/phi/Documents/COSIPY/cosipy-master/')
        os.system("python COSIPY.py")
        
        
        def mb_timeseries(cosipy_output,stakes_info_csv,stakes_data_file,position):

            pos = pd.read_csv(position,index_col='id')

            #Create output surface mb
            output_path = cosipy_output
            output = xr.open_dataset(output_path)

            MB = output['MB'] 

            #Read obs data
            stakes_loc_file = pd.read_csv(stakes_info_csv)  
            stakes_data_file = pd.read_csv(stakes_data_csv,index_col='TIMESTAMP')

            #Read periods of analysis
            dates = pd.to_datetime(stakes_data_file.index.tolist())

            stakes_summary = pd.DataFrame()

            for i in range(len(dates)):

                #Define anaylis period in function of observations
                if i == 0:
                    period_MB = MB.sel(time=slice(pd.to_datetime(output.time[0].values), dates[i]))
                else:
                    period_MB = MB.sel(time=slice(dates[i-1], dates[i]))

                #Calculate
                acum_period_MB = period_MB.sum(dim='time')

                #Obtain the stakes values from the nc    
                stakes_results = []
                stakes_names = []

                for s in range(pos.shape[0]):
                    stake_MB = round(float(acum_period_MB[pos.iloc[s,0],pos.iloc[s,1]].values),2)
                    stakes_results.append(stake_MB)
                if i == 0:
                    stakes_summary = pd.concat([stakes_summary, pd.DataFrame(stakes_results)], ignore_index = False)
                else:
                    stakes_summary[str(dates[i])] = stakes_results

            stakes_summary = stakes_summary.T
            stakes_summary.index = pd.to_datetime(dates)
            stakes_summary.columns = pos.index.to_list()
            stakes_summary.index.name = 'TIMESTAMP'
            stakes_summary.to_csv('/home/phi/Documents/Mocho-COSIPY/evaluation/bm_sim_'+stakes_data_csv[len(stakes_data_csv)-13:-4]+'.csv') #TOKEN
            
            output.close()

            return stakes_summary
        
        pos = '/home/phi/Documents/Mocho-COSIPY/input/stakes/position_2022-2023.csv' #TOKEN
        cosipy_output = '/home/phi/Documents/Mocho-COSIPY/output/output_mocho-choshuenco_20221014-20230313.nc' #TOKEN
        stakes_info_csv = '/home/phi/Documents/Mocho-COSIPY/input/stakes/stakes_info.csv' #TOKEN
        stakes_data_csv = '/home/phi/Documents/Mocho-COSIPY/input/stakes/stakes_data_2022-2023.csv' #TOKEN

        mb_timeseries(cosipy_output,stakes_info_csv,stakes_data_csv,pos)
        
        sim_path = '/home/phi/Documents/Mocho-COSIPY/evaluation/bm_sim_2022-2023.csv' #TOKEN
        
        sim = eval_to_cosipy(sim_path)
        sim = sim.iloc[:,0].values

        return sim

class spotpy_setup(object):  
    def __init__(self):
        self.cosipymodel = cosipy_model()

        # model parameters to calibrate #('parameter',min_value,max_value)
        #lower bound, upper bound, step size, initial value
        self.params = [spotpy.parameter.Uniform('albedo_fresh_snow',0.81,0.95),
                       spotpy.parameter.Uniform('albedo_firn',0.41,0.8),
                       spotpy.parameter.Uniform('albedo_ice',0.2,0.4),
                       spotpy.parameter.Uniform('albedo_mod_snow_aging',1,28),
                       spotpy.parameter.Uniform('albedo_mod_snow_depth',1,30),
                       spotpy.parameter.Uniform('roughness_fresh_snow',0.19,0.29),
                       spotpy.parameter.Uniform('roughness_ice',0.7,5),
                       spotpy.parameter.Uniform('roughness_firn',0.7,6.5)]

        return
    
    def parameters(self): 
        return spotpy.parameter.generate(self.params)

    def simulation(self,vector): 
        simulations = self.cosipymodel.run_cosipy(albedo_fresh_snow=vector[0],albedo_firn=vector[1],albedo_ice=vector[2],albedo_mod_snow_aging=vector[3],albedo_mod_snow_depth=vector[4],roughness_fresh_snow=vector[5],roughness_ice=vector[6],roughness_firn=vector[7])
        return simulations
    
    def evaluation(self,evaldates=False): #self,evaldates=False
        self.cosipymodel.get_obs()
        return self.cosipymodel.observations
        
    def objectivefunction(self,simulation,evaluation):
        objectivefunction = spotpy.objectivefunctions.rmse(simulation,evaluation)
        return objectivefunction
        

def calibrate():
    # calibration setup object
    outCal = '/home/phi/Documents/Mocho-COSIPY/evaluation/SCEUA-RMSE-2022-2023' 
    # initialize calibration algorithm with
    sampler = spotpy.algorithms.sceua(spotpy_setup(),dbname=outCal,dbformat='csv') 
    results = [] # mpty list to append iteration results !!! 
    # run calibration process
    sampler.sample(2) 
    
    results.append(sampler.getdata()) 

    print("---------------------------- PSANDOVALQ -------------------------------")
    print("-----------  https://github.com/SQPaul/COSIPY_Mocho-glacier -----------")
    return

    
if __name__ == "__main__":
    calibrate()